In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import json
import torch
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from skimage.transform import resize

from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
from params import *
from utils.plot import visualize_preds_indexed, visualize_preds
from utils.metrics import boxes_f1_score, precision_calc, get_boxes_from_df

## Load

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')
folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")
df_train['truth'] = (df_train['impact'] == 1) & (df_train['confidence'] > 1) & (df_train['visibility'] > 0) 


df_val = df_train[df_train["val_idx"] == 0]

In [ ]:
videos = df_val['video'].unique()

In [ ]:
# PREDS_PATH = OUT_DIR + '21_12/'
# epoch = 11
# preds = pd.read_csv(PREDS_PATH + f"pred_0_fold_epoch_{epoch}_score_001.csv")

PREDS_PATH = OUT_DIR + '22_12/'
epoch = 9
preds = pd.read_csv(PREDS_PATH + f"pred_0_fold_epoch_{epoch}_score_001_aug.csv")

In [ ]:
preds = preds[preds['pred'] > 0.1].reset_index(drop=True)

### Params

In [ ]:
# Probability thresholding
THRESHOLD_PRED = 0.8

# Adjacency post-processing
NMS_THRESHOLD = 0.35
THRESHOLD_IOU = 0.35
MAX_DIST = 4
MIN_CLUST_SIZE = 0

# View post-processing
MIN_DIST = 6

# Impact post-processing
MAX_FRAME_DIST = 10

## Score

In [ ]:
pred_val = preds[preds['pred'] > THRESHOLD_PRED].reset_index(drop=True)

In [ ]:
# scores = pred_val.groupby('video').agg(list)['pred'][videos].tolist()

In [ ]:
gt_boxes = get_boxes_from_df(df_val[df_val['truth'] == 1], videos)
pred_boxes = get_boxes_from_df(pred_val, videos)

In [ ]:
score = boxes_f1_score(pred_boxes, gt_boxes)

print(f' -> CV score is {score:.4f}')

### Post-processing

In [ ]:
from post_processing.adjacency import post_process_adjacency

In [ ]:
df_pred_pp = pred_val.copy()

In [ ]:
df_pred_pp = post_process_adjacency(
    df_pred_pp,
    threshold=THRESHOLD_IOU,
    max_dist=MAX_DIST,
    min_clust_size=MIN_CLUST_SIZE,
)

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

# Classifier inference

In [ ]:
from inference.classifier import *

### Data

In [ ]:
df_pred = preds.copy()

In [ ]:
df_pred['image_name'] = (df_pred['video'].str.replace('.mp4', '') + '_' +
                              df_pred['frame'].apply(lambda x: f'{x:04d}') + '.png')

In [ ]:
df_pred_vid = df_pred[df_pred["video"] == df_pred['video'].unique()[1]]

dataset = NFLDatasetClsInference(
    df_pred_vid,
    transforms=get_transfos_cls(train=False, visualize=True),
    root=IMG_PATH_F,
)

In [ ]:
# for i in range(10):
#     plt.imshow(dataset[i].numpy().transpose(1, 2, 0))
#     plt.show()

### Main

In [ ]:
images = df_pred["image_name"].unique()

In [ ]:
CP_FOLDER = "../dataset/"

In [ ]:
configs = {
    "efficientnet-b1": {
        "name": "efficientnet-b1",
        "num_classes": 1,
        "k": 5,
    },
    "efficientnet-b2": {
        "name": "efficientnet-b2",
        "num_classes": 1,
        "k": 5,
    },
    "efficientnet-b3": {
        "name": "efficientnet-b3",
        "num_classes": 1,
        "k": 5,
    },
    "efficientnet-b4": {
        "name": "efficientnet-b4",
        "num_classes": 1,
        "k": 5,
    },
    "resnet18": {
        "name": "resnet18",
        "num_classes": 1,
        "k": 5,
    },
    "resnet34": {
        "name": "resnet34",
        "num_classes": 1,
        "k": 5,
    },
}

In [ ]:
models = []
for model in configs:
    models += [retrieve_model(configs[model], fold=0, log_folder=CP_FOLDER)]

In [ ]:
preds = []
for vid in tqdm(df_pred['video'].unique()):
    df_pred_vid = df_pred[df_pred["video"] == vid]

    pred = inference(df_pred_vid, models, root=IMG_PATH_F)
    preds.append(pred)

In [ ]:
df_pred['pred_cls'] = np.concatenate(preds)

# Classifier with Aux inference

In [ ]:
from inference.classifier_aux import *

In [ ]:
CP_FOLDER = "../dataset2/"

In [ ]:
configs = {
    "efficientnet-b1": {
        "name": "efficientnet-b1",
        "num_classes": 1,
        "num_classes_aux": 4,
        "k": 5,
    },
    "efficientnet-b3": {
        "name": "efficientnet-b3",
        "num_classes": 1,
        "num_classes_aux": 4,
        "k": 5,
    },

    "resnet18": {
        "name": "resnet18",
        "num_classes": 1,
        "num_classes_aux": 4,
        "k": 5,
    },
}

In [ ]:
models = []
for model in configs:
    models += [retrieve_model_aux(configs[model], fold=0, log_folder=CP_FOLDER)]

In [ ]:
preds = []
preds_aux = []

for vid in tqdm(df_pred['video'].unique()):
    df_pred_vid = df_pred[df_pred["video"] == vid]

    pred, pred_aux = inference_aux(df_pred_vid, models, root=IMG_PATH_F)
    
    preds.append(pred)
    preds_aux.append(pred_aux)
    
#     break

In [ ]:
preds = np.concatenate(preds).flatten()
preds_aux = np.concatenate(preds_aux)

In [ ]:
df_pred['pred_aux_helmet'] = preds_aux[:, 1]
df_pred['pred_aux_body'] = preds_aux[:, 2]
df_pred['pred_aux_ground'] = preds_aux[:, 3]

impact_types = ["helmet", "body", "ground"]
df_pred['predicted_impact_type'] = [impact_types[c] for c in np.argmax(preds_aux[:, 1:], 1)]

In [ ]:
Counter(df_pred['predicted_impact_type'])

In [ ]:
df_pred['pred_cls_aux'] = preds

# Post-processing

In [ ]:
from post_processing.adjacency import post_process_adjacency
from post_processing.expansion import expand_boxes
from post_processing.view import post_process_view
from post_processing.pairing import post_process_pairing

### Params

In [ ]:
THRESHOLD_DET = 0.5
THRESHOLD_CLS = 0.5
MIN_CLUST_SIZE = 0

THRESHOLD_IOU = 0.35
MAX_DIST = 4

MIN_DIST = 5

R = 0.3

### Thresholding

In [ ]:
df_pred_pp = df_pred.copy()
df_pred_pp = df_pred_pp[df_pred_pp['pred'] > THRESHOLD_DET]
df_pred_pp = df_pred_pp[df_pred_pp['pred_cls'] > THRESHOLD_CLS]
df_pred_pp = df_pred_pp[df_pred_pp['pred_cls_aux'] > 0.25]

# df_pred_pp = df_pred_pp[
#     (df_pred_pp['pred_aux_helmet'] > 0.1) | (df_pred_pp['pred_aux_body'] > 0.1) | (df_pred_pp['pred_aux_ground'] > 0.1)
# ]

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

In [ ]:
Counter(df_pred_pp["predicted_impact_type"])

### Box expansion

In [ ]:
df_pred_pp = expand_boxes(df_pred_pp, r=R)

### Adjacency Post-processing

In [ ]:
df_pred_pp = post_process_adjacency(
    df_pred_pp,
    threshold=THRESHOLD_IOU,
    max_dist=MAX_DIST,
    min_clust_size=MIN_CLUST_SIZE,
)

In [ ]:
# df_pred_pp= df_pred_pp[df_pred_pp['predicted_impact_type'] != "ground"]

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

In [ ]:
Counter(df_pred_pp["predicted_impact_type"])

### View PP

In [ ]:
if MIN_DIST > 0:
    df_pred_pp_view = post_process_view(
        df_pred_pp, 
        min_dist=MIN_DIST,
#         keep_ground=False
    )

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp_view, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

In [ ]:
Counter(df_pred_pp_view["predicted_impact_type"])

### Helmet impact

In [ ]:
candidates = df_pred.copy()

In [ ]:
# for video in df_pred_pp_view['video'].unique()[3:4]:
#     df_vid = df_pred_pp_view[df_pred_pp_view['video'] == video]
    
#     df_vid_matched = post_process_helmet_impact(
#         df_vid, 
#         candidates,
#         alpha=10,
#         max_box_dist=100,
#         max_frame_dist=8, 
#         r=R,
#         max_dist=50,
#         cls_threshold=0.3,
#         det_threshold=0.,
#         verbose=1
#     )
    
#     idx = 0
#     frames = df_vid_matched['frame'].unique()
#     for frame in frames:
#         df_viz = df_vid_matched[df_vid_matched['frame'] == frame].reset_index(drop=True)
#         plt.figure(figsize=(16, 8))
#         visualize_preds_indexed(df_viz, 0, IMG_PATH_F, start_idx=idx)
#         idx += len(df_viz)
#         plt.axis(False)
#         plt.show()
    

In [ ]:
df_pred_pp_i = post_process_pairing(
    df_pred_pp_view,
    candidates,
    alpha=10, 
    max_box_dist=50, 
    max_frame_dist=4, 
    r=R,
    max_dist=40, 
    cls_threshold=0.,
    det_threshold=0.3,
    remove_unpaired=True,
)

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp_i, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

In [ ]:
Counter(df_pred_pp_i["predicted_impact_type"])

## Visualize results

In [ ]:
def get_match(df_video):
    cols = ['frame', "left", "width", "top", "height"]
    df_video['match'] = 0
    
    p_df = df_video[df_video['gt'] == 0].reset_index(drop=True)
    t_df = df_video[df_video['gt'] == 1].reset_index(drop=True)

    p = p_df[cols].values
    t = t_df[cols].values
    p[:, 2] += p[:, 1]
    p[:, 4] += p[:, 3]
    t[:, 2] += t[:, 1]
    t[:, 4] += t[:, 3]
    t = t[:, [0, 1, 3, 2, 4]]
    p = p[:, [0, 1, 3, 2, 4]]

    cost_matix, row_ind, col_ind = precision_calc(t, p, return_assignment=True)
    
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i, j] == 0:
            p_df.loc[j, 'match'] = 1
            t_df.loc[i, 'match'] = 1

    return pd.concat([p_df, t_df], 0).drop_duplicates().sort_values('frame').reset_index(drop=True)

In [ ]:
from utils.plot import plot_bboxes_pred

In [ ]:
def visualize_preds(df_pred, video_name, frame, root="", truth_col="impact", threshold_pred=0.7):
    img = f"{video_name[:-4]}_{frame:04d}.png"
    img = cv2.imread(root + img)

    df = df_pred[df_pred["video"] == video_name]
    df = df[df["frame"] == frame].reset_index(drop=True)

    try:
        boxes = df[["left", "width", "top", "height"]].values
    except KeyError:
        boxes = df[["x", "w", "y", "h"]].values

    boxes[:, 1] += boxes[:, 0]
    boxes[:, 3] += boxes[:, 2]
    boxes = boxes[:, [0, 2, 1, 3]]

    try:
        labels = [f"{l}\n{s:.3f}" for s, l in df[["pred", "predicted_impact_type"]].values]
    except KeyError:
        labels = [f"{s:.3f}" for s in df["pred"].values]
        labels = [l if l != "nan" else "" for l in labels]

    colors = []
    if "match" in df.columns:
        for truth, match in df[["gt", "match"]].values:
            if truth:
                if match:
                    colors.append("g")
                else:
                    colors.append("r")
            else:
                if match:
                    colors.append("b")
                else:
                    colors.append("orange")
    else:
        for pred, truth in df[["pred", truth_col]].values:
            if truth:
                if pred > threshold_pred:
                    colors.append("g")
                else:
                    colors.append("r")
            else:
                colors.append("b")

    plot_bboxes_pred(img, boxes, labels, colors)
    plt.title(f"Video {video_name} - frame {frame}")


In [ ]:
df_results = df_pred_pp_i.copy()

In [ ]:
df_truth = df_val[df_val['truth'] == 1]
df_truth = df_truth.rename(columns={"x": "left", "w": "width", "y": "top", "h":"height"})

In [ ]:
for idx, vid in enumerate(df_results['video'].unique()):
    df_vid_pred = df_results[df_results['video'] == vid].copy()
    df_vid_truth = df_truth[df_truth['video'] == vid].copy()
    df_vid_truth['gt'] = 1
    df_vid_pred['gt'] = 0
    
    df_vid = pd.concat([df_vid_truth, df_vid_pred])
    df_vid = get_match(df_vid)
    
    frames = sorted(df_vid['frame'].unique())
    for frame in frames:
        df_viz = df_vid[df_vid['frame'] == frame].reset_index(drop=True)
        plt.figure(figsize=(16, 8))
        
        visualize_preds(
            df_vid, 
            vid, 
            frame, 
            root=IMG_PATH_F, 
            truth_col="truth",
        )

        plt.axis(False)
        plt.show()

    if idx > 1:
        break